In [1]:
import argparse
import os
import pathlib
import sys
import uuid

import duckdb
import pandas as pd
from arg_parsing_utils import parse_args
from cytotable import convert, presets
from notebook_init_utils import bandicoot_check, init_notebook
from parsl.config import Config
from parsl.executors import HighThroughputExecutor

root_dir, in_notebook = init_notebook()

profile_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot")).resolve(), root_dir
)

In [2]:
if not in_notebook:
    args = parse_args()
    well_fov = args["well_fov"]
    patient = args["patient"]
    image_based_profiles_subparent_name = args["image_based_profiles_subparent_name"]

else:
    patient = "NF0014_T1"
    well_fov = "G2-2"
    image_based_profiles_subparent_name = "image_based_profiles"

In [3]:
input_sqlite_file = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/0.converted_profiles/{well_fov}/{well_fov}.duckdb"
).resolve(strict=True)
destination_sc_parquet_file = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/0.converted_profiles/{well_fov}/sc_profiles_{well_fov}.parquet"
).resolve()
destination_organoid_parquet_file = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/0.converted_profiles/{well_fov}/organoid_profiles_{well_fov}.parquet"
).resolve()
destination_sc_parquet_file.parent.mkdir(parents=True, exist_ok=True)
dest_datatype = "parquet"

In [4]:
# show the tables
with duckdb.connect(input_sqlite_file) as con:
    tables = con.execute("SHOW TABLES").fetchdf()
    print(tables)
    nuclei_table = con.sql("SELECT * FROM Nuclei").df()
    cells_table = con.sql("SELECT * FROM Cell").df()
    cytoplasm_table = con.sql("SELECT * FROM Cytoplasm").df()
    organoid_table = con.sql("SELECT * FROM Organoid").df()

        name
0       Cell
1  Cytoplasm
2     Nuclei
3   Organoid


In [5]:
nuclei_id_set = set(nuclei_table["object_id"].to_list())
cells_id_set = set(cells_table["object_id"].to_list())
cytoplasm_id_set = set(cytoplasm_table["object_id"].to_list())
# find the intersection of the three sets
intersection_set = nuclei_id_set.intersection(cells_id_set, cytoplasm_id_set)
# keep only the rows in the three tables that are in the intersection set
nuclei_table = nuclei_table[nuclei_table["object_id"].isin(intersection_set)]
cells_table = cells_table[cells_table["object_id"].isin(intersection_set)]
cytoplasm_table = cytoplasm_table[cytoplasm_table["object_id"].isin(intersection_set)]

In [6]:
# connect to DuckDB and register the tables
with duckdb.connect() as con:
    con.register("nuclei", nuclei_table)
    con.register("cells", cells_table)
    con.register("cytoplasm", cytoplasm_table)
    # Merge them with SQL
    merged_df = con.execute("""
        SELECT *
        FROM nuclei
        LEFT JOIN cells USING (object_id)
        LEFT JOIN cytoplasm USING (object_id)
    """).df()

In [7]:
# save the organoid data as parquet
print(f"Final organoid data shape: {merged_df.shape}")
organoid_table.to_parquet(destination_organoid_parquet_file, index=False)
organoid_table.head()

Final organoid data shape: (6, 1926)


,object_id,image_set,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_CENTER.X,Area.Size.Shape_Organoid_CENTER.Y,Area.Size.Shape_Organoid_CENTER.Z,Area.Size.Shape_Organoid_BBOX.VOLUME,Area.Size.Shape_Organoid_MIN.X,Area.Size.Shape_Organoid_MAX.X,Area.Size.Shape_Organoid_MIN.Y,...,Texture_Organoid_Mito_Difference.Entropy_256.3,Texture_Organoid_Mito_Difference.Variance_256.3,Texture_Organoid_Mito_Entropy_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_Mito_Inverse.Difference.Moment_256.3,Texture_Organoid_Mito_Sum.Average_256.3,Texture_Organoid_Mito_Sum.Entropy_256.3,Texture_Organoid_Mito_Sum.Variance_256.3,Texture_Organoid_Mito_Variance_256.3
0,12,G2-2,817009.0,639.377686,631.823242,6.351460,1299584.0,455.0,807.0,475.0,...,0.250958,0.003711,0.360444,-0.538834,0.480463,0.976973,2.598071,0.315261,271.487951,72.599963
1,14,G2-2,73845.0,657.491882,615.195435,13.476972,133536.0,484.0,796.0,506.0,...,0.030544,0.003873,0.036044,-0.225404,0.062927,0.997663,0.168499,0.032847,13.220485,4.741441


In [8]:
# drop columns that end with _x or _y lowercase
merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith(("_x", "_y"))]

In [9]:
print(f"Final merged single cell dataframe shape: {merged_df.shape}")
# save the sc data as parquet
merged_df.to_parquet(destination_sc_parquet_file, index=False)
merged_df.head()

Final merged single cell dataframe shape: (6, 1926)


,object_id,image_set,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_CENTER.X,Area.Size.Shape_Nuclei_CENTER.Y,Area.Size.Shape_Nuclei_CENTER.Z,Area.Size.Shape_Nuclei_BBOX.VOLUME,Area.Size.Shape_Nuclei_MIN.X,Area.Size.Shape_Nuclei_MAX.X,Area.Size.Shape_Nuclei_MIN.Y,...,Texture_Cytoplasm_Mito_Difference.Entropy_256.3,Texture_Cytoplasm_Mito_Difference.Variance_256.3,Texture_Cytoplasm_Mito_Entropy_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_Mito_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_Mito_Sum.Average_256.3,Texture_Cytoplasm_Mito_Sum.Entropy_256.3,Texture_Cytoplasm_Mito_Sum.Variance_256.3,Texture_Cytoplasm_Mito_Variance_256.3
0,18,G2-2,104.0,1213.250000,112.250000,1.000000,210.0,1206.0,1221.0,106.0,...,0.002945,0.003890,0.003647,-0.506467,0.049625,0.999831,0.028972,0.003399,4.908393,1.469928
1,24,G2-2,83.0,1436.759033,235.313248,1.000000,110.0,1432.0,1443.0,231.0,...,0.001159,0.003891,0.001346,-0.372497,0.024656,0.999940,0.010095,0.001265,1.814046,0.611963
2,143,G2-2,4220.0,600.478210,584.200684,6.532228,8200.0,578.0,619.0,560.0,...,0.012250,0.003885,0.014326,-0.329040,0.073214,0.999240,0.090188,0.013210,11.629497,3.928547
3,217,G2-2,20979.0,625.778320,635.671936,9.966109,38760.0,591.0,659.0,609.0,...,0.020813,0.003881,0.024451,-0.333728,0.096488,0.998672,0.230024,0.022264,44.323263,14.461634
4,242,G2-2,32017.0,739.666138,658.512512,7.098198,74304.0,694.0,780.0,615.0,...,0.034173,0.003873,0.042031,-0.422395,0.147734,0.997658,0.346184,0.038138,54.566021,16.417926
